In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# *Grounding* usando a Vertex AI Gemini API

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/intro_multimodal_rag.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Execute no Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/intro_multimodal_rag.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> Veja no GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/intro_multimodal_rag.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Abra no Workbench da Vertex AI
    </a>
  </td>
</table>


## Visão geral

O *Grounding* permite usar modelos de texto generativos para gerar conteúdo baseado em seus próprios documentos e dados. Esse recurso permite que o modelo acesse informações em tempo de execução que vão além dos dados de treinamento. Ao basear as respostas do modelo nos resultados da Pesquisa Google ou nos armazenamentos de dados no Vertex AI Search, os LLMs baseados em dados podem produzir respostas mais precisas, atualizadas e relevantes.

O *grounding* oferece os seguintes benefícios:

- Reduz as alucinações do modelo (casos em que o modelo gera conteúdo que não é factual)
- Ancora respostas do modelo a informações, documentos e fontes de dados específicas
- Aumenta a confiabilidade, precisão e aplicabilidade do conteúdo gerado

No contexto de aterramento na Vertex AI, é possível configurar duas fontes diferentes de aterramento:

1. Resultados da Pesquisa Google para dados disponíveis publicamente e indexados
2. Armazenamentos de dados no Vertex AI Search, que podem incluir seus próprios dados na forma de dados de sites, dados não estruturados ou dados estruturados

**NOTA**: Alguns dos recursos deste notebook de exemplo exigem acesso antecipado a determinados recursos por meio de uma lista de permissões. O Grounding with Vertex AI Search está disponível na visualização pública, enquanto o Grounding with Google Web Search está disponível na visualização privada. Para solicitar acesso antecipado aos recursos da visualização privada, entre em contato com o representante da sua conta ou com o suporte do Google Cloud.

### Objetivos

Neste tutorial, você aprende como:

- Gere respostas de modelo de bate-papo e texto LLM com base nos resultados da Pesquisa Google
- Compare os resultados de respostas LLM não fundamentadas com respostas LLM fundamentadas
- Crie e use um armazenamento de dados no Vertex AI Search para basear respostas em documentos e dados personalizados
- Gere respostas de modelo de chat e texto LLM com base nos resultados do Vertex AI Search

Este tutorial usa os seguintes serviços e recursos de IA do Google Cloud:
- Vertex AI
- Pesquisa e conversação da Vertex AI

As etapas executadas incluem:

- Configurando o LLM e solicitando vários exemplos
- Envio de prompts de exemplo para modelos generativos de texto e chat na Vertex AI
- Configurar um armazenamento de dados no Vertex AI Search com seus próprios dados
- Envio de prompts de exemplo com vários níveis de aterramento (sem aterramento, aterramento na web, aterramento no armazenamento de dados)

### Custos

Este tutorial usa os seguintes componentes de Google Cloud que podem gerar custos em sua fatura:

- Vertex AI

Saiba mais sobre [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing) e use a [calculadora de preços](https://cloud.google.com/products/calculator/) para gerar uma estimativa de custo com base no uso projetado.

## Primeiros passos

### Instale a SDK da Vertex AI
**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# ! pip3 install --upgrade --user google-cloud-aiplatform pymupdf

### **Reinicie o kernel do seu jupyter notebook** 

Como a instalação está sendo realizada com a opção `--user` é necessário reiniciar o kernel para que os novos módulos se tornem acessíveis.

**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ O kernel do notebook está sendo reiniciado. Por favor aguarde este processo ser finalizado antes de continuar com os próximos passos. ⚠️</b>
</div>

### **Somente para uso no Colab - Autentique o seu ambiente de notebook** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para realizar a autenticação da sua sessão de notebook com a Google Cloud Esse passo é importante **para utilização no Colab** para garantir que as chamadas a APIs de Google Cloud funcionem sem problemas.

In [ ]:
# import sys

# # Additional authentication is required for Google Colab
# if "google.colab" in sys.modules:
#     # Authenticate user to Google Cloud
#     from google.colab import auth

#     auth.authenticate_user()

### **Somente para uso no Colab - defina o projeto Google Cloud a ser utilizado** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para definir qual projeto Google Cloud será utilizado pelo Colab na execução deste notebook. Senão, siga para as próximas instruções.

In [ ]:
# if "google.colab" in sys.modules:
#     # Define project information
#     PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
#     LOCATION = "us-central1"  # @param {type:"string"}

#     # Initialize Vertex AI
#     import vertexai

#     vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importe as bibliotecas necessárias

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

from IPython.display import Markdown, display
from vertexai.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part
)

from vertexai.preview.generative_models import (
    Tool, 
    grounding
)

PROJECT_ID = "lucianomartins-demos-345000"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

## Importe o modelo `Gemini 1.0 Pro`

O Gemini Pro (`gemini-1.0-pro`) ajuda na realização de tarefas utilizando linguagem natural, chats multiturno de texto e código e para a geração de código.

In [ ]:
model = GenerativeModel("gemini-1.0-pro-vision")

## Exemplo: baseando-se nos resultados da Pesquisa Google

Neste exemplo, você comparará respostas LLM sem fundamento com respostas baseadas nos resultados de uma Pesquisa Google. Você fará uma pergunta sobre um lançamento recente de hardware da Google Store.

In [ ]:
prompt = (
    "Quais são o preço, cores disponíveis e opções de armazenamento do último Pixel Tablet lançado?"
)

## Texto gerado sem *grounding*

In [ ]:
response = model.generate_content(prompt)

display(Markdown(response.candidates[0].text))

## Geração de texto com base nos resultados da Pesquisa Google

Agora você pode adicionar a palavra-chave de ferramentas arg com uma ferramenta de *grounding* `grounding.GoogleSearchRetrieval()` para instruir o LLM a primeiro realizar uma pesquisa no Google com o prompt e, em seguida, construir uma resposta com base nos resultados da pesquisa na web:

In [ ]:
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())
response = model.generate_content(prompt, tools=[tool])
display(Markdown(response.candidates[0].text))
response